In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import sqlite3

import requests
from bs4 import BeautifulSoup
from time import sleep
import pickle
import os

In [ ]:
from scraper.footballsquads_scraper import Footballsquads_scraper
from database_io.db_handler import DB_handler

In [ ]:
dbh = DB_handler("/home/morten/Develop/packing-report/gde/GDE.db")
fh = Footballsquads_scraper("/home/morten/Develop/packing-report/gde/.cache_footballsquads/", dbh)

In [ ]:
fh.cache_to_db(leagues=["GER-Bundesliga", "GER-Bundesliga2", "ENG-Premier League"])

In [ ]:
fh.scrape_archive()

In [ ]:
table_html = fh.scrape_kit_number_table("http://www.footballsquads.co.uk/eng/2002-2003/conf/scar.htm")
table_dict = fh.extract_numbers_from_html_table(table_html)

In [ ]:
file = open("/home/morten/Develop/packing-report/Goal-Difference-Elo-GDE/.cache_footballsquads/ger-2021-2022-gerbun-monchen.pckl",'rb')
to = pickle.load(file)
file.close()

In [ ]:
def get_table(url):
    all_rows = []
    # Send a GET request to the URL
    response = requests.get("http://www.footballsquads.co.uk/ger/2021-2022/bundes/monchen.htm")

    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")

        # Find the table you want to scrape by inspecting the HTML source
        table = soup.find("div", {"id": "main"})

        if table:
            # Extract table rows
            rows = table.find_all("tr")

            # Loop through rows and extract data
            for row in rows:
                # Extract table data from each row
                cells = row.find_all(["th", "td", "i"], recursive=True)
                row_data = []
                for cell in cells:
                    # Check if the cell contains italic text
                    italic_text = cell.find("i")
                    if italic_text:
                        # If italic text is present, add it to row_data
                        row_data.append(italic_text.get_text(strip=True))
                    else:
                        # If no italic text, add the regular text
                        row_data.append(cell.get_text(strip=True))
                # Print row data
                all_rows.append(row_data)
        else:
            print("Table not found on the page.")
        print(all_rows)
    else:
        print("Failed to fetch the page. Status code:", response.status_code)

In [ ]:
scrape_url = f"http://www.footballsquads.co.uk/ger/2021-2022/bundes/monchen.htm"
team_name = "monchen"
file_path = f"ger-bundes-2020-2021-{team_name}.pckl"
if file_path not in os.listdir("/home/morten/Develop/packing-report/Goal-Difference-Elo-GDE/.cache_footballsquads"):
    print("Scraping ", scrape_url)
else:
    print("Data found in cache, skip")
data = get_table(scrape_url)
if not data:
    print("None Data found!")
with open("/home/morten/Develop/packing-report/Goal-Difference-Elo-GDE/.cache_footballsquads/" + file_path, 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
sleep(1)

In [ ]:
data='''<table>
<tr>
   <td>10</td>
   <td>Thuram</td>
</tr>
<tr>
   <td>11</td>
   <td>
      <i>Wolf </i>  
   </td>
</tr>
</table>'''


soup=BeautifulSoup(data,'html.parser')

for child in soup.find('table').findChildren("tr" , recursive=False):
  tdlist = []
  if child.find('table'):
     for td in child.findChildren("td", recursive=False):
         print(td.next_element.strip())
         for td1 in td.findChildren("table", recursive=False):
             for child1 in td1.findChildren("tr", recursive=False):
                 for child2 in child1.findChildren("td", recursive=False):
                     tdlist.append(child2.text)
                 print(' '.join(tdlist))
                 print(child2.next_element.next_element.strip())
  else:
     for td in child.findChildren("td" , recursive=False):
         tdlist.append(td.text)
     print(' '.join(tdlist))